In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals
from rasa_sdk import Action
from rasa_sdk.events import SlotSet
import zomatopy
import json
import requests
import ast

config={"user_key":"7f90aaa7bde29e0522cb3c9762c564c4"}
location = 'Mumbai'
cuisine = 'chinese'
average_cost_for_two = 2500
restaurantList=[]


global response
response=''
zomato = zomatopy.initialize_app(config)
location_detail=zomato.get_location(location, 1)
location_detail_json = json.loads(location_detail)

latitude=location_detail_json["location_suggestions"][0]["latitude"]
longitude=location_detail_json["location_suggestions"][0]["longitude"]

cuisines_dict={'bakery':5,'chinese':25,'cafe':30,'italian':55,'biryani':7,'north indian':50,'south indian':85}
results=zomato.restaurant_search("", latitude, longitude, str(cuisines_dict.get(cuisine)),20)
d = json.loads(results)
response=""
if d['results_found'] == 0:
    response= "no results"
else:
    for restaurant in d['restaurants']:
        restaurant_json =  '{ "name":"'+restaurant['restaurant']['name']+'", "address":"'+restaurant['restaurant']['location']['address']+'", "user_rating":"'+restaurant['restaurant']['user_rating']['aggregate_rating']+'", "user_rating":"'+restaurant['restaurant']['user_rating']['aggregate_rating']+'", "average_cost_for_two":"'+str(restaurant['restaurant']['average_cost_for_two'])+'"}'
        restaurantList.append(json.loads(restaurant_json))
        
filteredList = list(map(lambda x:x if int(x['average_cost_for_two']) <= average_cost_for_two else '',restaurantList))
while("" in filteredList) : 
    filteredList.remove("") 
    

response=response+str(len(filteredList))+" Restaurants Found(s)\n"
for count,restaurant in enumerate(filteredList):
    if( count < 10):
        response=response+"\n "+str(count+1)+". "+restaurant['name'] +" in " +restaurant['address']+" has been rated "+restaurant['user_rating']+" and average price is Rs "+restaurant['average_cost_for_two']+" for two people. \n"

print(response)

#Send Mail
import smtplib 
from email.mime.text import MIMEText
#email = tracker.get_slot('email')
email = 'jr.gauravsingh@gmail.com'
# for slack handling
s = smtplib.SMTP('smtp.gmail.com', 587) 
s.starttls() 
s.login('shikhacomputergeek@gmail.com','testingchatbot@1234')

message=MIMEText("The details of all the restaurants you inquried \n \n" + response+"\n\n Bon Appetite.")
message['From']="shikhacomputergeek@gmail.com"
message['To']=str(email)
message['Subject']='Foodie Restaurant Bot'
try:
    print('dd')
    s.sendmail("shikhacomputergeek@gmail.com",str(email), message.as_string())
    s.quit()
except Exception as e:
    print(e)
response = ""

20 Restaurants Found(s)

 1. Hitchki in 5, Transocean, Hiranandani Business Park, Lake Boulevard Road, Powai, Mumbai has been rated 4.9 and average price is Rs 1600 for two people. 

 2. The Fatty Bao in G1, 108, Ground Floor, Morya Classic, Opposite Infinity Mall, Off Link Road, Andheri Lokhandwala, Andheri West, Mumbai has been rated 4.9 and average price is Rs 1900 for two people. 

 3. Palladium Social in Level 3, Palladium Mall, Senapati Bapat Marg, Lower Parel, Mumbai has been rated 4.9 and average price is Rs 1400 for two people. 

 4. HeadQuarters in 166, 1st Floor, Opposite Regal Cinema, MG Road, Colaba, Mumbai has been rated 4.9 and average price is Rs 1800 for two people. 

 5. The Fatty Bao in Ground Floor, Summerville, Junction of 14th and 33rd Road, Linking Road, Bandra West, Mumbai has been rated 4.8 and average price is Rs 2000 for two people. 

 6. Indiana Water's in 2nd Floor, Seawoods Grand Central Mall, Seawoods Station Road, Sector 40, Nerul, Navi Mumbai has been r